In [1]:
midifile = 'data/chopin-fantaisie.mid'

In [185]:
import time
import copy
import subprocess
import numpy as np
from abc import abstractmethod

# Midi file parser
import midi

# Midi Playback
import pygame
import pygame.midi
from pygame.mixer import music


Init Pygame and Audio
--------

In [3]:
pygame.init()
pygame.midi.init()
# RUN timidity -iA !!
for id in xrange(pygame.midi.get_count()):
    print 'device', id, pygame.midi.get_device_info(id)
MIDI_DEVICE = 2

device 0 ('ALSA', 'Midi Through Port-0', 0, 1, 0)
device 1 ('ALSA', 'Midi Through Port-0', 1, 0, 0)
device 2 ('ALSA', 'TiMidity port 0', 0, 1, 0)
device 3 ('ALSA', 'TiMidity port 1', 0, 1, 0)
device 4 ('ALSA', 'TiMidity port 2', 0, 1, 0)
device 5 ('ALSA', 'TiMidity port 3', 0, 1, 0)


Midi Pattern
--------

In [153]:
def copy_event(e):
    '''
    Deep copy of midi.Event and related
    This is a hack; it would be better to have a copy()
    
    Test
    ----
    >>> n = midi.NoteOnEvent()
    >>> m = copy_event(n)
    >>> m.set_velocity(23)
    >>> m.get_velocity() == n.get_velocity()
    '''
    if isinstance(e, midi.Event):
        new_e = e.copy()
        new_e.data = list(new_e.data)
        return new_e
    elif isinstance(e, midi.AbstractEvent):
        new_e = e.__class__(tick=e.tick, data=list(e.data))
        return new_e
    else:
        raise ValueError('could not clone {}'.format(e))
        

class MidiPattern(midi.Pattern):
    # Types of event
    #
    # Important events are:
    # NoteOnEvent
    # NoteOffEventevents
    # ControlChangeEvent
    # ProgramChangeEvent
    # EndOfTrackEvent
    # SetTempoEvent
    # KeySignatureEvent
    #
    # Warning: NoteOnEvent with velocity 0 === NoteOffEvent
    
    def __init__(self, pattern):
        '''
        Build from midi.Pattern object
        '''
        if not isinstance(pattern, midi.Pattern):
            raise ValueError

        # clone or build
        copy_tracks = [midi.Track([copy_event(e) for e in track]) 
                       for track in pattern]
        midi.Pattern.__init__(self, tracks=copy_tracks, 
                              resolution=pattern.resolution, 
                              format=pattern.format, 
                              tick_relative=pattern.tick_relative)
        
        # copy attributes if needed
        if 'attributes' in pattern.__dict__:
            self.attributes = copy.deepcopy(pattern.attributes)
        
    def zero(self):
        '''
        Zero the tracks
        '''
        self[:] = []
        return self
        
    def init_attributes(self):
        '''
        Init attributes associated to events
        '''
        # add attributes
        self.attributes = [[{} for e in track] for track in self]
        
    def stamp_time(self, label, bpm=None):
        '''
        Add timestamp to each note's attribute,
        accounting for changes in tempo
        '''
        if not 'attributes' in self.__dict__:
            self.init_attributes()
        # Default Bpm is 120
        # If bpm is given, override all bpm changes
        fixed_bpm = (bpm is not None)
        bpm = 120. if bpm is None else float(bpm)
        for track, track_attributes in zip(self, self.attributes):
            total_time = 0.
            for e, e_attr in zip(track, track_attributes):
                dt = 60. / bpm / self.resolution * e.tick
                total_time += dt
                e_attr[label] = total_time

                if (isinstance(e, midi.SetTempoEvent) and not fixed_bpm):
                    bpm = e.get_bpm()
                    
    def sort_all(self):
        '''
        Jointly sort events and attributes, in-place
        '''
        was_relative = simple.tick_relative
        self.make_ticks_abs()
        for track, track_attributes in zip(self, self.attributes):
            new_track, new_track_attr = zip(*sorted(zip(track, track_attributes), key=lambda (a, b): a.tick))
            track[:] = midi.Track(new_track, tick_relative=False)
            track_attributes[:] = new_track_attr
        if was_relative:
            self.make_ticks_rel()
                    
    def play(self, bpm=None, instrument=None, verbose=False):
        '''
        Play pattern if in midi-0 format
        
        Parameters
        ----------
        bpm : Number, optional
            beats per minute (quarter notes)
            if given, tempo will be forced to bpm
        instrument : int, optional
            instrument in GM-1 table
            if not given, use given instruments
        '''
        '''
        if len(self) != 1 or self.format != 0:
            raise Exception('need midi-1')
        '''
        midi_player = pygame.midi.Output(MIDI_DEVICE)
        if instrument is not None:
            for ch in xrange(16):
                midi_player.set_instrument(instrument, ch)
        try:
            simple = self.simplified(bpm)
            # Default Bpm is 120
            # If bpm is given, override all bpm changes
            fixed_bpm = (bpm is not None)
            bpm = 120. if bpm is None else float(bpm)
            total_time = 0.
            for note_idx, note in enumerate(simple[0]):
                tick = note.tick
                dt = 60. / bpm / self.resolution * tick
                total_time += dt
                time.sleep(dt)
                if isinstance(note, midi.NoteEvent):
                    pitch = note.get_pitch()
                    velocity = note.get_velocity()
                    midi_player.note_on(pitch, velocity)
                elif (isinstance(note, midi.SetTempoEvent) and
                    not fixed_bpm):
                    bpm = note.get_bpm()
                    if verbose: print 'bpm change:', bpm
                elif (isinstance(note, midi.ProgramChangeEvent) and
                      instrument is None):
                    if verbose: print note
                    midi_player.set_instrument(note.value, note.channel)
        except KeyboardInterrupt:
            print 'Was playing note', note_idx, 'time', total_time
        finally:
            del midi_player
            
    def fix_bpm(self, bpm):
        if not self.tick_relative:
            raise Exception('convert to relative ticks first')
        tempo_event = midi.SetTempoEvent()
        tempo_event.set_bpm(bpm)
        # filter out tempo events
        for track in self:
            track[:] = midi.Track([e for e in track
                  if not isinstance(e, midi.SetTempoEvent)], 
                                  tick_relative=False)
        # add single tempo
        self[0].insert(0, tempo_event)
            
    def simplified(self, bpm=None):
        '''
        Simplify midi pattern by keeping only important events.
        Merge all tracks to one (midi-0 convention)

        Parameters
        ----------
        bpm : Number, optional
            beats per minute (quarter notes)
            if given, tempo will be forced to bpm [TODO],
            and all tempo events will be dropped

        Events Kept:
            midi.NoteEvent
            #midi.EndOfTrackEvent (only the last one is kept)
            midi.SetTempoEvent (if bpm is not given)
            midi.ProgramChangeEvent
            
        Returns
        -------
        simple : MidiEvent
            simplified MidiEvent
        '''
        # Change ticks to absolute and sort
        tmp = MidiPattern(self)
        tmp.make_ticks_abs()
        events = []
        for track in tmp:
            for e in track:
                events.append(e)
        events.sort()

        # Filter
        new_track = midi.Track([e for e in events
                  if isinstance(e, midi.NoteEvent) or 
                  isinstance(e, midi.ProgramChangeEvent) or
                  (isinstance(e, midi.SetTempoEvent) and
                   not bpm)], tick_relative=False)

        # Keep last end of track event
        end_of_track = max(e for e in events 
                           if isinstance(e, midi.EndOfTrackEvent))
        new_track.append(end_of_track)

        # To pattern, change to relative
        new_pattern = MidiPattern(self).zero()
        new_pattern.append(new_track)
        new_pattern.make_ticks_rel()

        # Fix bpm if needed
        if bpm:
            new_pattern.fix_bpm(bpm)
        
        return new_pattern

In [66]:
pattern = MidiPattern(midi.read_midifile(midifile))
simple = pattern.simplified(bpm=160)
simple.stamp_time('t0')
midi.write_midifile("generated/simple.mid", simple)

In [67]:
print simple.attributes[0][-40:]

[{'t0': 203.24999999999994}, {'t0': 203.25468749999993}, {'t0': 203.34843749999993}, {'t0': 203.34843749999993}, {'t0': 203.43749999999994}, {'t0': 203.44218749999993}, {'t0': 203.53124999999994}, {'t0': 203.53124999999994}, {'t0': 203.62499999999994}, {'t0': 203.62499999999994}, {'t0': 203.71874999999994}, {'t0': 203.71874999999994}, {'t0': 203.81249999999994}, {'t0': 203.81249999999994}, {'t0': 203.90624999999994}, {'t0': 203.90624999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 204.04687499999994}, {'t0': 204.09374999999994}, {'t0': 204.14062499999994}, {'t0': 204.18749999999994}, {'t0': 204.28124999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.54687499999994}, {'t0': 205.59374999999994}, {'t0': 205.64062499999994}, {'t0': 206.99999999999994}, {'t0': 206.99999999

In [68]:
pattern[0]
pattern.play(180)

Was playing note 28 time 3.77777777778


In [69]:
simple.play()

Was playing note 5 time 3.0


Distorter
--------

In [154]:
'''
TODO
add/drop notes
replace notes

WARNING
if elements are inserted or deleted,
need to rebuild attributes!!
'''
class Distorter(object):
    '''
    Distort a *simplified* pattern.
    
    Ways of distorting a pattern include changing its tempo,
    introducing randomness on the timing of individual notes (ticks),
    or changing the instruments (channel).
    
    No need to preserve measures.
    '''
    def distort(self, pattern, keep_stamps=False):
        '''
        Distort a pattern.
        
        Decorates _distort:
        - create new_pattern from pattern
        - add attributes and timestamps to new_pattern
        - call _distort()
        - add new timestamps to new_pattern
        
        
        Parameters
        ----------
        pattern : MidiPattern
            *simplified* pattern
        keep_stamps : bool
            if True, keep original time stamps 't0'
            typically, when applying a chaing of distortions,
            keep stamps except for the first distortion
            
        Returns
        -------
        new_pattern : MidiPattern
            new distorted pattern
        align : list
            alignment of new_pattern to input pattern
        '''
        new_pattern = MidiPattern(pattern)
        if not keep_stamps:
            new_pattern.init_attributes()
            new_pattern.stamp_time('t0')
        new_pattern = self._distort(pattern, new_pattern)
        new_pattern.sort_all()
        new_pattern.stamp_time('t')
        return new_pattern
    
    @abstractmethod
    def _distort(self, pattern, new_pattern):
        '''
        Actual implementation. Modifies new_pattern in place.
        
        Parameters
        ----------
        pattern : MidiPattern
            input pattern
        new_pattern : MidiPattern
            pattern to return
        '''
        pass
    
    @abstractmethod
    def __repr__(self):
        pass
    
    def randomize(self, params):
        '''
        Sample parameters of distorter.
        
        Parameters
        ----------
        params : dict
            hyperparameters defining distribution on parameters
        '''
        pass
    
    def __repr__(self):
        name = self.__class__.__name__
        params = ['{}={}'.format(k,v) for k,v in self.__dict__.items()]
        return '<{}({})>\n'.format(name, ','.join(params))
    

class VelocityNoiseDistorter(Distorter):
    '''
    Add gaussian noise on individual velocities
    '''
    def __init__(self, sigma=10.):
        '''
        Parameters            
        
        ----------
        sigma : float
            standard deviation of gaussian noise
        '''
        self.sigma = sigma
        
    def __repr__(self):
        return 'VelocityNoiseDistorter(sigma={:.2f})'.format(self.sigma) 
            
    def randomize(self, params=None):
        '''
        Parameters
        ----------
        params : dict
            min_sigma, max_sigma : float
                where self.sigma ~ U(min_sigma, max_sigma)
        '''
        p = {'min_sigma': 0., 'max_sigma': 20.}
        if params:
            p.update(params)
        self.sigma = np.random.uniform(
            p['min_sigma'], p['max_sigma'])
        
    def _distort(self, pattern, new_pattern):
        for track in new_pattern:
            for e in track:
                if (isinstance(e, midi.NoteOnEvent) and
                        e.get_velocity() > 0):
                    tmp_velocity = e.get_velocity() + self.sigma*np.random.normal() 
                    e.set_velocity(np.clip(int(tmp_velocity), 1, 127))
        return new_pattern

    
class VelocityWalkDistorter(Distorter):
    '''
    Generate bounded random walk,
    and multiply velocities by it.
    '''
    def __init__(self, sigma=10., min=0.5, max=2.):
        '''
        Parameters
        ----------
        sigma : float
            standard deviation of derivative of random walk
            per quarter note
        min : float
            minimum multiple of original velocity
        max : float
            maximum multiple of original velocity
        '''
        self.sigma = sigma
        self.min = min
        self.max = max
        
    def __repr__(self):
        return 'VelocityWalkDistorter(sigma={:.2f}, min={:.2f}, max={:.2f})'.format(self.sigma, self.min, self.max) 
    
    def randomize(self, params=None):
        '''
        Parameters
        ----------
        params : dict
            min_sigma, max_sigma : float
                where self.sigma ~ U(min_sigma, max_sigma)
            min, max : float
                where self.min, self.max ~ U(min, max)
                and min < max
        '''
        p = {'min_sigma': 0., 'max_sigma': 20.,
             'min': 0.3, 'max': 1.5}
        if params:
            p.update(params)
        self.sigma = np.random.uniform(p['min_sigma'], p['max_sigma'])
        a, b = np.random.uniform(p['min'], p['max'], size=2)
        self.min = min(a, b)
        self.max = max(a, b)
        
    def _distort(self, pattern, new_pattern):
        sigma_per_tick = self.sigma / np.sqrt(pattern.resolution)
        multiple = 1.  # original value
        for track in new_pattern:
            for e in track:
                tmp_multiple = (multiple + sigma_per_tick 
                                * np.random.normal()
                                * np.sqrt(float(e.tick)))
                multiple = np.clip(tmp_multiple, self.min, self.max)
                #print multiple
                if (isinstance(e, midi.NoteOnEvent) and
                        e.get_velocity() > 0):
                    tmp_velocity = e.get_velocity() * multiple
                    e.set_velocity(np.clip(int(tmp_velocity), 1, 127))
        return new_pattern
    
    
class ProgramDistorter(Distorter):
    '''
    Change Instrument
    '''
    def __init__(self, ticks=0):
        '''
        Parameters
        ----------
        ticks : int
            change instrument every ticks
        '''
        self.ticks = ticks
        
    def __repr__(self):
        return 'ProgramDistorter(ticks={:.2f})'.format(self.ticks)
    
    def randomize(self, params=None):
        '''
        For now, just sample an instrument from the list at random.
        
        Parameters
        ----------
        params : dict
            instruments : list of int
                list of instruments to sample from
        
        [TODO]
        Parameters
        ----------
        params : dict
            lambda : float
                define distribution on number of instruments self.ni
                where self.ni ~ Exp(lambda)
                i.e., P(self.ni)
            min, max : float
                where self.min, self.max ~ U(min, max)
                and min < max
        '''
        p = {'instruments': [0, 1, 2, 3]}
        if params:
            p.update(params)
        self.instrument = np.random.choice(p['instruments'])
        
    def _distort(self, pattern, new_pattern):
        new_pattern.zero()
        new_events = [midi.ProgramChangeEvent(
                channel=ch,
                value=self.instrument) for ch in xrange(16)]
        for idx, e in enumerate(pattern[0]):
            if not isinstance(e, midi.ProgramChangeEvent):
                new_events.append(e)
                '''
                if idx % self.ticks == 0:
                    instrument = np.random.randint(4)
                    new_events += [midi.ProgramChangeEvent(
                        channel=ch,
                        value=instrument) for ch in xrange(16)]
                '''
        new_pattern.append(midi.Track(new_events))
        # Rebuild attributes and stamps
        new_pattern.init_attributes()
        new_pattern.stamp_time('t0')
        return new_pattern
    
    
class TempoDistorter(Distorter):
    '''
    Change tempo by offsetting ticks.
    
    This ignores SetTempoEvent and does not introduce additional ones.
    Instead it offsets the ticks.
    '''
    def __init__(self, sigma=0.5, min=0.5, max=2.):
        '''
        Parameters
        ----------
        sigma : float
            standard deviation of derivative of random walk
            per quarter note
        min : float
            minimum multiple of original tempo
        max : float
            maximum multiple of original tempo
        '''
        self.sigma = sigma
        self.min = min
        self.max = max
        
    def __repr__(self):
        return 'TempoDistorter(sigma={:.2f}, min={:.2f}, max={:.2f})'.format(self.sigma, self.min, self.max) 
    
    def randomize(self, params=None):
        '''
        Parameters
        ----------
        params : dict
            min_sigma, max_sigma : float
                where self.sigma ~ U(min_sigma, max_sigma)
            min, max : float
                where self.min, self.max ~ U(min, max)
                and min < max
        '''
        p = {'min_sigma': 0., 'max_sigma': 1.,
             'min': 0.6, 'max': 1.5}
        if params:
            p.update(params)
        self.sigma = np.random.uniform(p['min_sigma'], p['max_sigma'])
        a, b = np.random.uniform(p['min'], p['max'], size=2)
        self.min = min(a, b)
        self.max = max(a, b)
        
    def _distort(self, pattern, new_pattern):
        sigma_per_tick = self.sigma / np.sqrt(pattern.resolution)
        multiple = 1.  # original value
        for track in new_pattern:
            for e in track:
                tmp_multiple = (multiple + sigma_per_tick 
                                * np.random.normal()
                                * np.sqrt(float(e.tick)))
                multiple = np.clip(tmp_multiple, self.min, self.max)
                #print 'multiple {} bpm {}'.format(
                    #multiple, 120./multiple)
                tmp_tick = e.tick * multiple
                e.tick = np.clip(int(tmp_tick), 1, 127)
        return new_pattern
    
    

class TimeNoiseDistorter(Distorter):
    '''
    Add gaussian noise on individual note event ticks
    TODO: make sigma relative to note duration,
          change note duration
    
    TODO: Special care must be taken to preserve NoteOn/NoteOff precedence.
    '''
    def __init__(self, sigma=0.1):
        '''
        Parameters
        ----------
        sigma : float
            standard deviation of offset on tick in quarter notes
        '''
        self.sigma = sigma
        
    def __repr__(self):
        return 'TimeNoiseDistorter(sigma={:.2f})'.format(self.sigma)
        
    def randomize(self, params=None):
        '''
        Parameters
        ----------
        params : dict
            min_sigma, max_sigma : float
                where self.sigma ~ U(min_sigma, max_sigma)
        '''
        p = {'min_sigma': 0., 'max_sigma': 0.1}
        if params:
            p.update(params)
        self.sigma = np.random.uniform(
            p['min_sigma'], p['max_sigma'])
        
    def _distort(self, pattern, new_pattern):
        new_pattern.make_ticks_abs()
        for track, track_attributes in zip(new_pattern, new_pattern.attributes):
            for e in track:
                if (isinstance(e, midi.NoteOnEvent) and
                        e.get_velocity() > 0):
                    tmp_tick = self.sigma*np.random.normal()*pattern.resolution
                    e.tick = max(int(e.tick + tmp_tick), 1)
            # Fix end of track event - make it the last
            end_of_track_tick = max(e.tick for e in track)
            for e in track:
                if isinstance(e, midi.EndOfTrackEvent):
                    e.tick = end_of_track_tick
        new_pattern.make_ticks_rel()
        return new_pattern



In [155]:
distorter = VelocityNoiseDistorter(sigma=20.)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-noise.mid', dist_pattern)
dist_pattern.play(bpm=180)

VelocityNoiseDistorter(sigma=5.55)
Was playing note 5 time 2.66666666667


In [87]:
print dist_pattern.attributes[0][-4:]

[{'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 209.52109374999995, 't0': 209.52109374999995}]


In [88]:
distorter = VelocityWalkDistorter(sigma=0.1)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-walk.mid', dist_pattern)
dist_pattern.play(bpm=180)

VelocityWalkDistorter(sigma=18.86, min=1.05, max=1.30)
Was playing note 5 time 2.66666666667


In [89]:
distorter = ProgramDistorter()
distorter.randomize()
# for some reason GM 1- 3 makes no sound in pygame?
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/program.mid', dist_pattern)
dist_pattern.play(bpm=180)

ProgramDistorter(ticks=0.00)
Was playing note 19 time 2.66666666667


In [135]:
distorter = TempoDistorter(sigma=0, min=0.5, max=2.)
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/tempo.mid', dist_pattern)
dist_pattern.play(bpm=180)

TempoDistorter(sigma=0.19, min=0.63, max=1.14)
time warp [{'t': 207.00000000000017, 't0': 206.99999999999994}, {'t': 207.00000000000017, 't0': 206.99999999999994}, {'t': 207.00000000000017, 't0': 206.99999999999994}, {'t': 209.52109375000018, 't0': 209.52109374999995}]
Was playing note 145 time 4.16805555556


In [91]:
distorter = TimeNoiseDistorter()
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/time.mid', dist_pattern)
dist_pattern.play(bpm=180)

TimeNoiseDistorter(sigma=0.04)
time warp [{'t': 149.29479166666837, 't0': 206.99999999999994}, {'t': 149.29557291666836, 't0': 206.99999999999994}, {'t': 149.29635416666835, 't0': 206.99999999999994}, {'t': 149.39557291666836, 't0': 209.52109374999995}]
Was playing note 5 time 2.66597222222


Individual Note Times to Global Alignment
-------

In [131]:
def align_frame_to_frame(pattern, stride):
    '''
    Parameters
    ----------
    pattern : MidiPattern
        pattern with alignment attributes
        't0' for reference time
        't' for candidate time (times to align)
    stride : float
        stride of window in seconds
        
    Returns
    -------
    align : list of int
        alignment of each candidate window to index of target window
        
    TODO
    ----
    Smooth using kernel instead of binning candidate + averaging target
    reference by chord/duration of nth note
    interpolate empty reference measures    
        
    Currently, move a non-overlapping window over the pattern.
    Candidate window events have their time averaged.
    It is then assigned to the closest window.
    '''
    # Collect info for each candidate window
    windows = {}
    for track_attributes in pattern.attributes:
        for e_attr in track_attributes:
            ref_idx = int(e_attr['t'] / stride)
            windows.setdefault(ref_idx, [])
            windows[ref_idx].append(e_attr['t0'])
    # Average
    avg = {}
    for cand_idx, w in windows.items():
        avg[cand_idx] = int(sum(t for t in w) / float(len(w)))
    # Fill holes
    align = []
    last = 0.
    for cand_idx, ref_idx in sorted(avg.items()):
        # fill holes with linear interpolation
        repl = np.linspace(float(last), float(ref_idx), num=(cand_idx - len(align) + 2))[1:]
        last = ref_idx
        #repl = [ref_idx] * (cand_idx - len(align) + 1)
        align += list(repl.astype(int))
    return align
        
    

In [137]:
stride = 1.
align = align_frame_to_frame(dist_pattern, stride)
align

[2,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 64,
 65,
 66,
 67,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 78,
 79,
 80,
 81,
 83,
 84,
 85,
 86,
 88,
 89,
 90,
 92,
 93,
 94,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 155,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 166,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 185,
 186,
 188,
 189,
 191,
 192,
 193,
 194,
 196,
 197,
 198,
 199,
 200,
 201,
 203,
 205]

Alignment IO
----

In [179]:
def write_align(fname, align, stride):
    '''
    Write alignment to file
    '''
    with open(fname, 'w') as f:
        f.write('{}\n'.format(stride))
        f.write('\n'.join(map(str, align)))
        
def read_align(fname):
    '''
    Read alignment from file\
    
    Returns
    -------
    align : list of float
        alignments in seconds for each candidate window
    stride : float
        duration of window
    '''
    with open(fname, 'r') as f:
        numbers = [float(l.strip()) for l in f]
        return numbers[1:], numbers[0]

In [181]:
write_align('generated/align.txt', align, stride)
align2, stride2 = read_align('generated/align.txt')
print align2 == align
print int(stride2) == int(stride), stride2, stride

True
True 1.0 1.0


In [157]:
def random_distort(pattern, distorters=None):
    '''
    Distort a simple pattern by applying a chain
    for distortions on it.
    
    Parameters
    ----------
    pattern : MidiPattern
        pattern to distort
    distorters : list of Distorter
        distorters to apply
    '''
    if not distorters:
        distorters = [TempoDistorter(), TimeNoiseDistorter()]
        for distorter in distorters:
            distorter.randomize()
    current = simple
    for i, distorter in enumerate(distorters):
        keep_stamps = i > 0
        current = distorter.distort(current, keep_stamps)
    return current



In [160]:
dist_pattern = random_distort(simple)
align = align_frame_to_frame(dist_pattern, stride=1.)
print align
dist_pattern.play()

[2, 4, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 123, 124, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 197, 198, 199, 200, 201, 202, 203, 205]
Was playing note 85 time 3.44088541667


Actual Generation
----

In [193]:
num_samples = 10
stride = 0.1
for i in xrange(num_samples):
    base_name = 'generated/sample-{}'.format(i)
    align_name = '{}.txt'.format(base_name)
    midi_name = '{}.mid'.format(base_name)
    wav_name = '{}.wav'.format(base_name)
    distorted = random_distort(base)
    align = align_frame_to_frame(distorted, stride)
    write_align(align_name, align, stride)
    midi.write_midifile(midi_name, distorted)
    # Convert to wav using timidity
    print wav_name
    subprocess.check_call(['timidity', '-Ow', midi_name, '-o', wav_name])
    print 'Done generating {}'.format(base_name)

generated/sample-0.wav
Done generating generated/sample-0
generated/sample-1.wav
Done generating generated/sample-1
generated/sample-2.wav
Done generating generated/sample-2
generated/sample-3.wav
Done generating generated/sample-3
generated/sample-4.wav
Done generating generated/sample-4
generated/sample-5.wav
Done generating generated/sample-5
generated/sample-6.wav
Done generating generated/sample-6
generated/sample-7.wav
Done generating generated/sample-7
generated/sample-8.wav
Done generating generated/sample-8
generated/sample-9.wav
Done generating generated/sample-9
